# PyCity Schools Analysis

* Your analysis here
---

In [18]:
# Dependencies and Setup
from pathlib import Path
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## Local Government Area Summary

In [19]:
# Calculate the Totals (Schools and Students)
school_count =len(school_data_complete["school_name"].unique())
student_count =len(school_data_complete.index)

# Calculate the Total Budget
total_budget =sum(school_data_complete["budget"].unique())


In [20]:
# Calculate the Average Scores
average_maths_score =school_data_complete["maths_score"].mean()
average_reading_score =school_data_complete["reading_score"].mean()


In [21]:
# Calculate the Percentage Pass Rates
passing_maths_count = school_data_complete[(school_data_complete["maths_score"] >= 50)].count()["student_name"]
passing_maths_percentage = passing_maths_count / float(student_count) * 100
passing_reading_count =school_data_complete[(school_data_complete["reading_score"] >= 50)].count()["student_name"]
passing_reading_percentage =passing_reading_count/float(student_count)*100

passing_maths_reading_count = school_data_complete[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)
].count()["student_name"]

overall_passing_rate =passing_maths_reading_count/float(student_count) * 100


In [22]:
# Convert to DataFrame
area_summary =pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Maths Score": [average_maths_score],
    "Average Reading Score": [average_reading_score],
    "%  Passing Math": [passing_maths_percentage],
    "%  Passing Reading": [passing_reading_percentage],
    "% Overall Passing":[overall_passing_rate]
})

# Formatting
area_summary["Total Students"] = area_summary["Total Students"].map("{:,}".format)
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
area_summary


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


In [23]:
#rename columns
school_data_complete.rename(columns = {'Student ID':'student_id', 'School ID':'school_id'}, inplace=True)

#total number of schools
total_schools = school_data_complete.school_name.nunique()

#total number of students
total_students = len(school_data_complete.student_id.unique())

#total budget
total_budget = school_data.budget.sum()

#average math score
avg_math_score = school_data_complete.maths_score.mean()

#average reading score
avg_reading_score = school_data_complete.reading_score.mean()

#lambda function to determine passing grade
passing = lambda x: "Pass" if x >= 50 else "Fail"

#create column for passing math
school_data_complete['passing_math'] = school_data_complete.maths_score.apply(passing)

#create column for passing reading
school_data_complete['passing_reading'] = school_data_complete.reading_score.apply(passing)

#create column for passing overall
school_data_complete['passing_overall'] = school_data_complete.apply(lambda row:
  'Pass'
  if row['passing_math'] == "Pass" and row['passing_reading'] == "Pass"
  else 'Fail',
  axis=1
)

#percentage of students with a passing math score, 70 or greater
#value count where to determine the amount of students who passed math
passed_math_count = school_data_complete['passing_math'].str.contains('Pass').value_counts()[True]

percent_passed_math = passed_math_count / total_students * 100

#percentage of students with a passing reading score, 70 or greater
passed_reading_count = school_data_complete['passing_reading'].str.contains('Pass').value_counts()[True]

percent_passed_reading = passed_reading_count / total_students * 100

#percentage of students who passed math and reading, 70 or greater
passed_overall_count = school_data_complete['passing_overall'].str.contains('Pass').value_counts()[True]

percent_passed_overall = passed_overall_count / total_students * 100

In [24]:
school_name_df = school_data_complete.groupby(['school_name'])

#school name variable, differentiated from column name
school_name_var = school_name_df['school_name']

#school type
#'.first()' method used to retrieve relevant data
school_type = school_name_df['type'].first()

#size method used due to groupby method
total_students_by_school = school_name_df.size()

#total students per school
students_per_school = student_data['school_name'].value_counts()

#budget
total_budget_per_school = (school_name_df['budget'].sum())

#school name size
school_name_size = school_name_df['size'].value_counts()

#budget per school
total_budget_schools = (total_budget_per_school/total_students_by_school)

#total budget per student
total_budget_per_student = total_budget_schools/students_per_school

#average math score
avg_math_score_by_school = school_name_df.maths_score.mean()

#average reading score
avg_reading_score_by_school = school_name_df.reading_score.mean()

#percentage of students with a passing math score, 70 or greater, grouped by school
passing_math_by_school = school_data_complete[school_data_complete['passing_math']=='Pass'].groupby(['school_name']).size()
percent_passing_math_by_school = (passing_math_by_school/students_per_school)*100

#percentage of students with a passing reading score, 70 or greater, grouped by school
passing_reading_by_school = school_data_complete[school_data_complete['passing_reading']=='Pass'].groupby(['school_name']).size()
percent_passing_reading_by_school = (passing_reading_by_school/students_per_school)*100

#percentage of students with a passing overall score, 70 or greater, grouped by school
passing_overall_by_school = school_data_complete[school_data_complete['passing_overall']=='Pass'].groupby(['school_name']).size()
percent_passing_overall_by_school = (passing_overall_by_school/students_per_school)*100



In [25]:
school_summary_table_data = {'School Type': school_type,
    'Total Students': students_per_school,
    'Total School Budget': total_budget_schools,
    'Per Student Budget': total_budget_per_student,
    'Average Math Score': avg_math_score_by_school,
    'Average Reading Score': avg_reading_score_by_school,
    '% Passing Math': percent_passing_math_by_school,
    '% Passing Reading': percent_passing_reading_by_school,
    '% Overall Passing Rate': percent_passing_overall_by_school,}
school_summary_df = pd.DataFrame(school_summary_table_data)
school_summary_df = school_summary_df[
                        ["School Type", "Total Students", "Total School Budget",
                         "Per Student Budget", "Average Math Score", "Average Reading Score",
                         "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
#school_summary_df["Total School Budget"]=school_summary_df["Total School Budget"].map("${:,.2f}".format)
#school_summary_df["Per Student Budget"]=school_summary_df["Per Student Budget"].map("${:,.2f}".format)


school_summary_df.head(15)



,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,Government,4976,3124928.0,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,1081356.0,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,1884411.0,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,1763916.0,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,917500.0,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,3022020.0,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,248087.0,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,1910635.0,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650.0,650.0,68.843100,69.039277,82.062592,81.978576,67.191766


## School Summary

## Top Performing Schools (By % Overall Passing)

In [26]:
# Sort and show top five schools
top_schools =school_summary_df.sort_values(["% Overall Passing Rate"], ascending=False)
top_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Griffin High School,Independent,1468,917500.0,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,1081356.0,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,3124928.0,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,1049400.0,583.0,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,2547363.0,637.0,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

In [27]:
# Sort and show bottom five schools
bottom_schools =school_summary_df.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Hernandez High School,Government,4635,3022020.0,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,1910635.0,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650.0,650.0,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,1319574.0,578.0,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,1763916.0,644.0,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

In [28]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores =school_data_complete.loc[school_data_complete["year"] == 9].groupby("school_name")["maths_score"].mean()
year_ten_scores =school_data_complete.loc[school_data_complete["year"] == 10].groupby("school_name")["maths_score"].mean()
year_eleven_scores =school_data_complete.loc[school_data_complete["year"] == 11].groupby("school_name")["maths_score"].mean()
year_twelve_scores =school_data_complete.loc[school_data_complete["year"] == 12].groupby("school_name")["maths_score"].mean()

# Combine series into single DataFrame
maths_scores_by_year =pd.concat([year_nine_scores, year_ten_scores, year_eleven_scores, year_twelve_scores], axis=1)


# Minor data wrangling
maths_scores_by_year.index.name = None
maths_scores_by_year.columns = ["9th Grade", "10th Grade", "11th Grade", "12th Grade"]
maths_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]] = maths_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]].applymap("{:,.6f}".format)

# Display the DataFrame
maths_scores_by_year


/var/folders/zs/hh5swvp55_sc2db9q_cg1c340000gn/T/ipykernel_88298/133784309.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  maths_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]] = maths_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]].applymap("{:,.6f}".format)


,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


## Reading Score by Year

In [29]:
# Create data series of scores by year levels using conditionals
year_nine = school_data_complete[(school_data_complete["year"] == 9)]
year_ten = school_data_complete[(school_data_complete["year"] == 10)]
year_eleven = school_data_complete[(school_data_complete["year"] == 11)]
year_twelve = school_data_complete[(school_data_complete["year"] == 12)]

# Group each by school name
year_nine_scores =school_data_complete.loc[school_data_complete["year"] == 9].groupby("school_name")["reading_score"].mean()
year_ten_scores =school_data_complete.loc[school_data_complete["year"] == 10].groupby("school_name")["reading_score"].mean()
year_eleven_scores =school_data_complete.loc[school_data_complete["year"] == 11].groupby("school_name")["reading_score"].mean()
year_twelve_scores =school_data_complete.loc[school_data_complete["year"] == 12].groupby("school_name")["reading_score"].mean()

# Combine series into single DataFrame
reading_scores_by_year =pd.concat([year_nine_scores,year_ten_scores,year_eleven_scores,year_twelve_scores], axis=1)


# Minor data wrangling
reading_scores_by_year.index.name = None
reading_scores_by_year.columns = ["9th Grade", "10th Grade", "11th Grade", "12th Grade"]
reading_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]] = reading_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]].applymap("{:,.6f}".format)

# Display the DataFrame
reading_scores_by_year


/var/folders/zs/hh5swvp55_sc2db9q_cg1c340000gn/T/ipykernel_88298/2674089402.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  reading_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]] = reading_scores_by_year[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]].applymap("{:,.6f}".format)


,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

In [30]:
# Establish the bins
spending_bins = [0, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]


In [31]:
# Create a copy of the school summary since it has the "Per Student Budget"
#  This step can be skipped but its best to make a copy.
school_spending_df = school_summary_df


In [32]:
scores_spending = school_summary_df.loc[:,["School Type", "Total Students", "Total School Budget",
                         "Per Student Budget",'Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate']]
# Add a new columns named Spending Ranges (Per Student) and binning based off budget per student
scores_spending['Spending Ranges (Per Student)']= pd.cut(school_summary_df['Per Student Budget'],spending_bins,labels=group_names)
scores_spending["Total School Budget"]=scores_spending["Total School Budget"].map("${:,.2f}".format)
scores_spending["Per Student Budget"]=scores_spending["Per Student Budget"].map("${:,.2f}".format)

# Create a group based off of the bins
scores_spending = scores_spending.groupby('Spending Ranges (Per Student)')
scores_spending.head()


/var/folders/zs/hh5swvp55_sc2db9q_cg1c340000gn/T/ipykernel_88298/1872224703.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  scores_spending = scores_spending.groupby('Spending Ranges (Per Student)')


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,$585-630
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,<$585
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,$630-645
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,$630-645
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,$585-630
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,$645-680
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,<$585
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,$645-680
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766,$645-680


## Scores by School Size

In [33]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [34]:
# Create a new data frame by locating the desired columns
scores_size = school_summary_df.loc[:,["School Type", "Total Students", "Total School Budget",
                         "Per Student Budget",'Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Add a new columns named School Size and binning based off total students
scores_size['School Size']= pd.cut(school_summary_df['Total Students'],size_bins,labels=group_names)
# Create a group based off of the bins
scores_size["Total School Budget"]=scores_size["Total School Budget"].map("${:,.2f}".format)
scores_size["Per Student Budget"]=scores_size["Per Student Budget"].map("${:,.2f}".format)

scores_size = scores_size.groupby('School Size')
scores_size.head()

/var/folders/zs/hh5swvp55_sc2db9q_cg1c340000gn/T/ipykernel_88298/1785118802.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  scores_size = scores_size.groupby('School Size')


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,School Size
school_name,,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405,Large (2000-5000)
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791,Medium (1000-2000)
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051,Large (2000-5000)
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880,Large (2000-5000)
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150,Medium (1000-2000)
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617,Large (2000-5000)
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717,Small (<1000)
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376,Large (2000-5000)
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979,Small (<1000)


## Scores by School Type

In [35]:
# Create new series using groupby for:
# Type | Average Maths Score | Average Reading Score | % Passing Maths | % Passing Reading | % Overall Passing

type_maths_scores = school_summary_df.groupby(["School Type"])["Average Math Score"].mean()
type_reading_scores = school_summary_df.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_maths = school_summary_df.groupby(["School Type"])["% Passing Math"].mean()
type_passing_reading = school_summary_df.groupby(["School Type"])["% Passing Reading"].mean()
type_overall_passing = school_summary_df.groupby(["School Type"])["% Overall Passing Rate"].mean()


In [36]:
scores_type = school_summary_df[['School Type','Average Math Score',
                                  'Average Reading Score','% Passing Math',
                                  '% Passing Reading','% Overall Passing Rate',]]
# Create a group based off of the school type
scores_type = scores_type.groupby('School Type').mean()
scores_type.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
